<a href="https://colab.research.google.com/github/deiveleal/data/blob/main/mestrado/ft105/classificacao/MultilayerPerceptronsClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#FT105A - Tópico Interdisciplinar I: Introdução ao Aprendizado de Máquina
### Aluno: Deive Audieres Leal
### RA: 083423


### Tarefa 3: Classification with Multilayer Perceptrons (MLP)

#### Link para o repositório: [MultilayerPerceptronsClassifier](https://github.com/deiveleal/data/blob/main/mestrado/ft105/classificacao/MultilayerPerceptrons-MLP.ipynbb)

### Enunciado:

Escolha uma das ferramentas computacionais  que tenha implementação de redes neurais do tipo MLP, e aplique tal algoritmo ao conjunto de dados Liver Disorder (http://goo.gl/QCuYTc);

* Utilize a mesma metodologia da atividade anterior, ou seja, subamostragem aleatória com 5 repetições, sendo 70% dos dados para treinamento e 30% dos dados para teste (em cada repetição);
* Faça todos os ajustes necessários no conjunto de dados para que as MLPs possam ser aplicadas (e descreva, no relatório, os ajustes feitos).
* Apresente o erro médio de classificação, para o conjunto de testes, e compare os resultados com os obtidos na atividade anterior.

Apresente claramente a metodologia adotada, os classificadores utilizados (versões e parâmetros) e discuta os resultados obtidos.

#### Instala o repositório da UCI

In [ ]:
!pip install ucimlrepo

#### Importação das bibliotecas

In [ ]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import naive_bayes as nb
from sklearn import neighbors
from ucimlrepo import fetch_ucirepo
from scipy.stats import mode
import numpy as np

#### Importa o conjunto de dados

#### Realiza a busca dos dados


In [ ]:
liver_disorders = fetch_ucirepo(id=60)

#### Os dados são um dataframe pandas já separado em features e targets (atributo alvo)

In [ ]:
X = liver_disorders.data.features.astype(str)
y = liver_disorders.data.targets.astype(str)

### Mostra o dataframe montado

In [ ]:
pd.concat([X,y], axis=1).head(7)

,mcv,alkphos,sgpt,sgot,gammagt,drinks
0,85,92,45,27,31,0.0
1,85,64,59,32,23,0.0
2,86,54,33,16,54,0.0
3,91,78,34,24,36,0.0
4,87,70,12,28,10,0.0
5,98,55,13,17,17,0.0
6,88,62,20,17,9,0.5


##### Cria função com o modelo de árvore de decisão. Foram usados os parâmetros padrões com exceção do criterio que passei a utilizar o 'log_loss' como ganho da informação ao invés do padrão gini. É retornado a acurácia e a previsão do modelo ao final da execução.

In [ ]:
def tree_classifier(X_train, X_test, y_train, y_test):
    clf = tree.DecisionTreeClassifier(criterion='log_loss')
    clf = clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    return {"accuracy":accuracy, "prediction":y_pred}

##### Cria função com o modelo knn. Foram usados os parâmetros padrões. É retornado a acurácia e a previsão do modelo ao final da execução.

In [ ]:
def knn_classifier(X_train, X_test, y_train, y_test):
    clf = neighbors.KNeighborsClassifier()
    clf = clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    return {"accuracy":accuracy, "prediction":y_pred}

##### Cria função com o modelo Naive Bayes. Foram usados os parâmetros padrões. É retornado a acurácia e a previsão do modelo ao final da execução.

In [ ]:
def nb_classifier(X_train, X_test, y_train, y_test):
    clf = nb.GaussianNB()
    clf = clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    return {"accuracy":accuracy, "prediction":y_pred}

##### Cria função que recebe os valores de predição e realiza a votação nos valores

In [ ]:
def ensemble_voting(predictions):
    predictions = [np.array(prediction).astype(float) for prediction in predictions]
    predictions_ensemble = mode(predictions, axis=0)[0]
    predictions_ensemble = np.squeeze(predictions_ensemble)
    return predictions_ensemble

#### Treina os classificadores e realiza a previsão com 5 rodadas de treino e classificação

In [ ]:
ensemble_source = {}
for execution in range(0, 5):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    ensemble_source[f"tree_accuracy_{execution}"] = tree_classifier(X_train, X_test, y_train, y_test)["accuracy"]
    ensemble_source[f"tree_classification_{execution}"] = tree_classifier(X_train, X_test, y_train, y_test)["prediction"]
    ensemble_source[f"knn_accuracy_{execution}"] = tree_classifier(X_train, X_test, y_train, y_test)["accuracy"]
    ensemble_source[f"knn_classification_{execution}"] = tree_classifier(X_train, X_test, y_train, y_test)["prediction"]
    ensemble_source[f"nb_accuracy_{execution}"] = tree_classifier(X_train, X_test, y_train, y_test)["accuracy"]
    ensemble_source[f"nb_classification_{execution}"] = tree_classifier(X_train, X_test, y_train, y_test)["prediction"]

#### Cria dataframe com os resultados da classificação

In [ ]:
df_ensemble = pd.DataFrame(ensemble_source)

#### Aplica a votação nos valores preditos

In [ ]:
predictions_ensemble = ensemble_voting([
    list(df_ensemble["tree_classification_0"]),
    list(df_ensemble["knn_classification_0"]),
    list(df_ensemble["nb_classification_0"]),
    list(df_ensemble["tree_classification_1"]),
    list(df_ensemble["knn_classification_1"]),
    list(df_ensemble["nb_classification_1"]),
    list(df_ensemble["tree_classification_2"]),
    list(df_ensemble["knn_classification_2"]),
    list(df_ensemble["nb_classification_2"]),
    list(df_ensemble["tree_classification_3"]),
    list(df_ensemble["knn_classification_3"]),
    list(df_ensemble["nb_classification_3"]),
    list(df_ensemble["tree_classification_4"]),
    list(df_ensemble["knn_classification_4"]),
    list(df_ensemble["nb_classification_4"])
])

#### Mostra os valores de previsão após votação

In [ ]:
pd.Series(predictions_ensemble, name="ensemble")

0       4.0
1       0.5
2       0.5
3      12.0
4       0.5
       ... 
99      2.0
100     0.5
101     6.0
102     0.5
103     0.5
Name: ensemble, Length: 104, dtype: float64